In [ ]:
# Caso esteja usando Google Colab, descomente a linha abaixo

# !pip install basedosdados

# Caso esteja utilizando o ambiente local, descomente as linhas abaixo

# import subprocess
# subprocess.run('pip install basedosdados', shell=True)

# Desafio Escritório de Dados

O desafio consiste em responder uma série de perguntas a partir da análise de 3 tabelas de dados. Como o programa faz download dos dados e guarda em memória, o mais eficiente seria realizar as analises em sql e só rodar o mínimo para diminuir o volume de dados.

Entretanto, na descrição é solicitado a utilização da biblioteca pandas. Dessa forma, iremos carregar as tabelas em memória para que possamos realizar as manipulações em pandas.

O único corte que será feito é selecionar apenas dados entre 01/01/2022 e 31/12/2023 da tabela de chamados.

In [42]:
#@title Configuring databases 💽
import basedosdados as bd
import pandas as pd

# selecionando dados entre 01/01/2022 e 31/12/2023 para diminuir a base para download
query_chamados = """
SELECT
  *
FROM datario.administracao_servicos_publicos.chamado_1746
WHERE data_inicio BETWEEN '2022-01-01' AND '2023-12-31'
"""

query_bairros = """
SELECT
  *
FROM datario.dados_mestres.bairro
"""

query_eventos = """
SELECT
  *
FROM datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos
"""

chamados = bd.read_sql(query_chamados, billing_project_id='emp-data-rio')


Downloading: 100%|██████████| 4/4 [00:00<00:00, 21.08rows/s]


In [96]:
bairros = bd.read_sql(query_bairros, billing_project_id='emp-data-rio')
eventos = bd.read_sql(query_eventos, billing_project_id='emp-data-rio')

eventos['data_inicial'] = pd.to_datetime(eventos['data_inicial'])
eventos['data_final'] = pd.to_datetime(eventos['data_final'])

Downloading: 100%|██████████| 4/4 [00:00<00:00, 18.78rows/s]


# Parte 1 - Localização de chamados do 1746

In [17]:
#@title Quantos chamados foram abertos no dia 01/04/2023?

chamados.loc[chamados['data_inicio'].astype(str).str[:10] == '2023-04-01'].shape[0]

73

In [21]:
#@title Qual o tipo de chamado que teve mais reclamações no dia 01/04/2023?

chamados.loc[chamados['data_inicio'].astype(str).str[:10] == '2023-04-01']\
  .groupby('tipo')\
  .size()\
  .sort_values(ascending=False)\
  .index[0]

'Poluição sonora'

In [31]:
#@title Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

chamados.merge(bairros, how="left", on="id_bairro")\
  .loc[chamados['data_inicio'].astype(str).str[:10] == '2023-04-01']\
  .groupby(['id_bairro', 'nome'])\
  .size()\
  .sort_values(ascending=False)\
  .head(3)

id_bairro  nome             
66         Engenho de Dentro    8
144        Campo Grande         6
26         Leblon               6
dtype: int64

In [33]:
#@title Qual o nome da subprefeitura com mais chamados abertos nesse dia?

chamados.merge(bairros, how="left", on="id_bairro")\
  .loc[chamados['data_inicio'].astype(str).str[:10] == '2023-04-01']\
  .groupby(['subprefeitura'])\
  .size()\
  .sort_values(ascending=False)\
  .index[0]

'Zona Norte'

In [38]:
#@title Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

chamados.merge(bairros, how="left", on="id_bairro")\
  .loc[
      (chamados['data_inicio'].astype(str).str[:10] == '2023-04-01') &
       (chamados['id_bairro'].isna() | bairros['subprefeitura'].isna())
       ]\
  .head()[['id_chamado', 'id_bairro', 'tipo', 'subtipo']]

,id_chamado,id_bairro,tipo,subtipo
1413418,18516246,None,Ônibus,Verificação de ar condicionado inoperante no ô...


Existe um chamado sem bairro associado. Isso provavelmente se dá ao tipo e subtipo do chamado, que é uma verificação de ar condicionado inoperante de um Ônibus, logo, não é necessáriamente ligado a um bairro.



# Parte 2 - Chamados do 1746 em grandes eventos

In [39]:
#@title Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

chamados.loc[
    (chamados['subtipo'] == 'Perturbação do sossego') &
    (chamados['data_inicio'].astype(str).str[:10] >= '2022-01-01') &
    (chamados['data_inicio'].astype(str).str[:10] <= '2023-12-31')
    ]\
    .shape[0]

42408

In [132]:
#@title Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

# primeiro, vamos manipular os DFs para facilitar o join, a solução é ineficiente, o ideal seria realizar o join direto no sql

eventos['intervalo'] = list(zip(eventos['data_inicial'], eventos['data_final']))
eventos['evento_id'] = eventos.index

bins = pd.IntervalIndex.from_tuples(eventos['intervalo'], closed='both')

# criando correspondencias para os intervalos
categorias = pd.cut(chamados['data_inicio'], bins)

# criando coluna de evento
chamados['evento_id'] = categorias.cat.codes

chamados_eventos = chamados.merge(eventos, how='left', on='evento_id')

In [141]:
#agora, vamos selecionar apenas aqueles com subtipo pertubação de sossego e nos eventos listados
filtro = (
    (chamados_eventos['evento_id'] != -1) &
    (chamados_eventos['subtipo'] == 'Perturbação do sossego') &
    (chamados_eventos['evento'].isin(['Reveillon', 'Carnaval', 'Rock in Rio']))
)

chamados_eventos\
    .loc[filtro][['id_chamado', 'evento', 'subtipo', 'data_inicio']]

,id_chamado,evento,subtipo,data_inicio
42130,18075942,Reveillon,Perturbação do sossego,2022-12-30 01:49:56
42134,18075963,Reveillon,Perturbação do sossego,2022-12-30 03:20:28
42138,18078383,Reveillon,Perturbação do sossego,2022-12-30 23:20:10
42938,18078426,Reveillon,Perturbação do sossego,2022-12-31 01:07:35
116880,17682439,Rock in Rio,Perturbação do sossego,2022-09-09 21:35:22
...,...,...,...,...
1603991,18328048,Carnaval,Perturbação do sossego,2023-02-18 09:48:20
1603999,18329661,Carnaval,Perturbação do sossego,2023-02-18 23:40:12
1604001,18330996,Carnaval,Perturbação do sossego,2023-02-19 19:38:37
1604004,18329734,Carnaval,Perturbação do sossego,2023-02-19 00:46:24


In [142]:
#@title Quantos chamados desse subtipo foram abertos em cada evento?

chamados_eventos\
    .loc[filtro]\
    .groupby(by=['evento'])\
    .size()\
    .sort_values(ascending=False)

evento
Rock in Rio    518
Carnaval       197
Reveillon       79
dtype: int64

In [160]:
#@title Qual evento teve a maior média diária de chamados abertos desse subtipo?

df_filtrado = chamados_eventos.loc[filtro]
df_filtrado['data_inicio_normalizada'] = df_filtrado['data_inicio'].dt.date

# Agrupa por 'evento' e calcula o total de chamados
total_chamados_por_evento = df_filtrado.groupby('evento')['id_chamado'].count()

# Conta o número de dias únicos para cada evento
dias_unicos_por_evento = df_filtrado.groupby('evento')['data_inicio_normalizada'].nunique()

# Calcula a média diária de chamados por evento
media_eventos = total_chamados_por_evento / dias_unicos_por_evento

media_eventos.sort_values(ascending=False).head(1)

evento
Rock in Rio    103.6
dtype: float64

In [162]:
#@title Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023

chamados['data_inicio_normalizada'] = chamados['data_inicio'].dt.date


filtro_periodo = chamados[
    (chamados['data_inicio'] >= '2022-01-01') &
     (chamados['data_inicio'] <= '2023-12-31') &
      (chamados['subtipo'] == 'Perturbação do sossego')]

media_total = filtro_periodo['id_chamado'].nunique() / filtro_periodo['data_inicio_normalizada'].nunique()
media_total


63.20119225037258

In [163]:
# comparação

media_eventos / media_total

evento
Carnaval       1.039010
Reveillon      0.624988
Rock in Rio    1.639210
dtype: float64